In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

In [4]:
data_train = np.load(r"datasets/Trousers & Jeans - All - Train.npz")
data_val = np.load(r"datasets/Trousers & Jeans - All - Validation.npz")
data_test = np.load(r"datasets/Trousers & Jeans - All - Test.npz")

In [5]:
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [6]:
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [7]:
# Defining the hypermatarest we would test and their range
HP_LAMBDA_REG = hp.HParam('lambda', hp.Discrete([0.0, 1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 0.1]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'D:/sam/Logs/Model 2/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_LAMBDA_REG],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [8]:
EPOCHS = 20
BATCH_SIZE = 64

In [9]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=(120,90,3), kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.Dense(4)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model with parameter value for the optimizer
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy', tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, name='sparse_categorical_crossentropy')])
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    
    # Logging the training process data to use later in tensorboard
    log_dir = "D:/sam/Logs/Model 2/fit/" + "run-{}".format(session_num)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')  
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male', 'Jenas Male', 'Trousers Female', 'Jeans Female'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the per-epoch callback.
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_sparse_categorical_crossentropy',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
    
    _, accuracy, _ = model.evaluate(images_val,labels_val)
    
    model.save(r"D:/sam/Logs/Model 2/Run-{}".format(session_num))
    
    return accuracy

In [10]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [11]:
session_num = 1

for lambda_reg in HP_LAMBDA_REG.domain.values:

    hparams = {
        HP_LAMBDA_REG: lambda_reg
    }
                    
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('D:/sam/Logs/Model 2/hparam_tuning/' + run_name, hparams, session_num)

    session_num += 1

--- Starting trial: run-1
{'lambda': 0.0}
Epoch 1/20
63/63 - 49s - loss: 1.2743 - accuracy: 0.5180 - sparse_categorical_crossentropy: 1.2713 - val_loss: 0.9768 - val_accuracy: 0.5880 - val_sparse_categorical_crossentropy: 0.9776
Epoch 2/20
63/63 - 47s - loss: 0.8401 - accuracy: 0.6501 - sparse_categorical_crossentropy: 0.8392 - val_loss: 0.8720 - val_accuracy: 0.6360 - val_sparse_categorical_crossentropy: 0.8746
Epoch 3/20
63/63 - 45s - loss: 0.7415 - accuracy: 0.7023 - sparse_categorical_crossentropy: 0.7424 - val_loss: 0.8666 - val_accuracy: 0.6480 - val_sparse_categorical_crossentropy: 0.8668
Epoch 4/20
63/63 - 45s - loss: 0.6641 - accuracy: 0.7378 - sparse_categorical_crossentropy: 0.6645 - val_loss: 0.8758 - val_accuracy: 0.6520 - val_sparse_categorical_crossentropy: 0.8763
Epoch 5/20
63/63 - 46s - loss: 0.6205 - accuracy: 0.7440 - sparse_categorical_crossentropy: 0.6207 - val_loss: 0.7691 - val_accuracy: 0.6900 - val_sparse_categorical_crossentropy: 0.7709
Epoch 6/20
63/63 - 47s 

Epoch 2/20
63/63 - 48s - loss: 1.1326 - accuracy: 0.6518 - sparse_categorical_crossentropy: 0.8657 - val_loss: 1.2268 - val_accuracy: 0.5680 - val_sparse_categorical_crossentropy: 0.9943
Epoch 3/20
63/63 - 50s - loss: 1.0141 - accuracy: 0.6713 - sparse_categorical_crossentropy: 0.7991 - val_loss: 1.1087 - val_accuracy: 0.6240 - val_sparse_categorical_crossentropy: 0.9091
Epoch 4/20
63/63 - 50s - loss: 0.9284 - accuracy: 0.6913 - sparse_categorical_crossentropy: 0.7403 - val_loss: 1.0061 - val_accuracy: 0.6540 - val_sparse_categorical_crossentropy: 0.8335
Epoch 5/20
63/63 - 49s - loss: 0.8736 - accuracy: 0.7113 - sparse_categorical_crossentropy: 0.7102 - val_loss: 0.9913 - val_accuracy: 0.6360 - val_sparse_categorical_crossentropy: 0.8354
Epoch 6/20
63/63 - 49s - loss: 0.7903 - accuracy: 0.7485 - sparse_categorical_crossentropy: 0.6392 - val_loss: 0.8789 - val_accuracy: 0.6900 - val_sparse_categorical_crossentropy: 0.7291
Epoch 7/20
63/63 - 49s - loss: 0.7596 - accuracy: 0.7617 - sparse

Epoch 14/20
63/63 - 46s - loss: 1.0697 - accuracy: 0.6556 - sparse_categorical_crossentropy: 0.8539 - val_loss: 1.1506 - val_accuracy: 0.6300 - val_sparse_categorical_crossentropy: 0.9367
Epoch 15/20
63/63 - 46s - loss: 1.0680 - accuracy: 0.6541 - sparse_categorical_crossentropy: 0.8591 - val_loss: 1.1587 - val_accuracy: 0.6040 - val_sparse_categorical_crossentropy: 0.9536
16/16 [==============================] - 2s 103ms/step - loss: 1.1114 - accuracy: 0.6120 - sparse_categorical_crossentropy: 0.9070
INFO:tensorflow:Assets written to: D:/sam/Logs/Model 2/Run-9\assets
--- Starting trial: run-10
{'lambda': 0.1}
Epoch 1/20
63/63 - 47s - loss: 12.8810 - accuracy: 0.3859 - sparse_categorical_crossentropy: 1.3885 - val_loss: 2.4780 - val_accuracy: 0.4960 - val_sparse_categorical_crossentropy: 1.2376
Epoch 2/20
63/63 - 45s - loss: 2.0048 - accuracy: 0.4930 - sparse_categorical_crossentropy: 1.1681 - val_loss: 1.8664 - val_accuracy: 0.4180 - val_sparse_categorical_crossentropy: 1.2732
Epoch 3

In [12]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 2/fit"

In [13]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 2/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
